This compares the custom gradient of the Woodbury form to that of an equivalent layer with no decomposition.

In [1]:
import tensorflow as tf
import numpy as np
import matrix_decompositions_tf as fctr
import transforms as transf
import math
import post_process_grad as ppg

fltrSz = (3,3)
fftSz = (8,8)
noc = 3
nof = 8
rho = 1.0
dtype = tf.complex128
nots = 100
novs = 100

AInv = fctr.dictionary_object2D_full(fltrSz=fltrSz,fftSz=fftSz,noc = noc,nof = nof,rho = rho,dtype=dtype)

wt = tf.complex(tf.random.normal((nots,) + fftSz + (noc,1),dtype=dtype.real_dtype),tf.random.normal((nots,) + fftSz + (noc,1),dtype = dtype.real_dtype))
wt = wt + tf.math.conj(tf.reverse(wt,axis=(2,3)))
wt = wt[slice(None),slice(None),slice(0,5,1),slice(None),slice(None)]
wv = tf.complex(tf.random.normal((novs,) + fftSz + (noc,1),dtype=dtype.real_dtype),tf.random.normal((novs,) + fftSz + (noc,1),dtype = dtype.real_dtype))
wv = wv + tf.math.conj(tf.reverse(wv,axis=(2,3)))
wv = wv[slice(None),slice(None),slice(0,5,1),slice(None),slice(None)]

B = transf.fft2d_inner(fftSz)(tf.random.normal((1,) + fltrSz + (noc,nof,),dtype=dtype.real_dtype))


xt = tf.linalg.matmul(B,wt,adjoint_a = True)
xv = tf.linalg.matmul(B,wv,adjoint_a = True)
idmat = tf.eye(nof,dtype=dtype,batch_shape = (int(tf.rank(B)) - 2)*(1,))
yt = tf.linalg.matmul(tf.linalg.inv(rho*idmat + tf.linalg.matmul(B,B,adjoint_a=True)),xt)
yv = tf.linalg.matmul(tf.linalg.inv(rho*idmat + tf.linalg.matmul(B,B,adjoint_a=True)),xv)

w = tf.keras.layers.Input(shape=(fftSz[0],math.floor(fftSz[1]/2.) + 1,) + (noc,1,),dtype=dtype)
x = tf.linalg.matmul(B,w,adjoint_a = True)
y1 = AInv.qinv(x)
y2 = fctr.QInv_auto(AInv.dhmul,rho)(x)

model1 = ppg.Model_record_grad(w,y2)
model1.compile(loss = tf.keras.losses.MSE,run_eagerly=True,optimizer=tf.keras.optimizers.SGD())
model1.fit(x=wt,y=yt,batch_size=10,epochs=8,shuffle=False,validation_data = (wv,yv))
model2 = ppg.Model_passenger(model1.gradient_record,w,y1)
model2.compile(loss = tf.keras.losses.MSE,run_eagerly=True,optimizer=tf.keras.optimizers.SGD())
model2.fit(x=wt,y=yt,batch_size=10,epochs=8,shuffle=False,validation_data = (wv,yv))
maxError = 0.
maxGrad = 0.
for grad1,grad2 in zip(model1.gradient_record,model2.gradient_record):
    for gradval1,gradval2 in zip(grad1,grad2):
        maxError = max(maxError,tf.math.reduce_max(tf.math.abs((gradval1 - gradval2)*tf.math.conj(gradval1 - gradval2))))
        maxGrad = max(maxGrad,tf.math.reduce_max(tf.math.abs(gradval1*tf.math.conj(gradval1))),tf.math.reduce_max(tf.math.abs(gradval2*tf.math.conj(gradval2))))

print(maxGrad)
print(maxError)

Epoch 1/8
10/10 [==============================] - 0s 15ms/step - loss: 64.7896 - val_loss: 63.8838
Epoch 2/8
10/10 [==============================] - 0s 14ms/step - loss: 64.0875 - val_loss: 63.2137
Epoch 3/8
10/10 [==============================] - 0s 14ms/step - loss: 63.3788 - val_loss: 62.5375
Epoch 4/8
10/10 [==============================] - 0s 14ms/step - loss: 62.6641 - val_loss: 61.8558
Epoch 5/8
10/10 [==============================] - 0s 14ms/step - loss: 61.9439 - val_loss: 61.1693
Epoch 6/8
10/10 [==============================] - 0s 15ms/step - loss: 61.2193 - val_loss: 60.4785
Epoch 7/8
10/10 [==============================] - 0s 15ms/step - loss: 60.4910 - val_loss: 59.7844
Epoch 8/8
10/10 [==============================] - 0s 17ms/step - loss: 59.7599 - val_loss: 59.0877
Epoch 1/8
10/10 [==============================] - 0s 22ms/step - loss: 64.7881 - val_loss: 63.8804
Epoch 2/8
10/10 [==============================] - 0s 20ms/step - loss: 64.0824 - val_loss: 63.2067


This cell compares the custom gradient for the standard form to that of an equivalent layer with automatic differentiation.

In [1]:
import tensorflow as tf
import numpy as np
import matrix_decompositions_tf as fctr
import transforms as transf
import math
import post_process_grad as ppg

fltrSz = (3,3)
fftSz = (8,8)
noc = 8
nof = 3
rho = 1.0
dtype = tf.complex128
nots = 100
novs = 100

AInv = fctr.dictionary_object2D_full(fltrSz=fltrSz,fftSz=fftSz,noc = noc,nof = nof,rho = rho,dtype=dtype)

wt = tf.complex(tf.random.normal((nots,) + fftSz + (noc,1),dtype=dtype.real_dtype),tf.random.normal((nots,) + fftSz + (noc,1),dtype = dtype.real_dtype))
wt = wt + tf.math.conj(tf.reverse(wt,axis=(2,3)))
wt = wt[slice(None),slice(None),slice(0,5,1),slice(None),slice(None)]
wv = tf.complex(tf.random.normal((novs,) + fftSz + (noc,1),dtype=dtype.real_dtype),tf.random.normal((novs,) + fftSz + (noc,1),dtype = dtype.real_dtype))
wv = wv + tf.math.conj(tf.reverse(wv,axis=(2,3)))
wv = wv[slice(None),slice(None),slice(0,5,1),slice(None),slice(None)]

B = transf.fft2d_inner(fftSz)(tf.random.normal((1,) + fltrSz + (noc,nof,),dtype=dtype.real_dtype))


xt = tf.linalg.matmul(B,wt,adjoint_a = True)
xv = tf.linalg.matmul(B,wv,adjoint_a = True)
idmat = tf.eye(nof,dtype=dtype,batch_shape = (int(tf.rank(B)) - 2)*(1,))
yt = tf.linalg.matmul(tf.linalg.inv(rho*idmat + tf.linalg.matmul(B,B,adjoint_a=True)),xt)
yv = tf.linalg.matmul(tf.linalg.inv(rho*idmat + tf.linalg.matmul(B,B,adjoint_a=True)),xv)

w = tf.keras.layers.Input(shape=(fftSz[0],math.floor(fftSz[1]/2.) + 1,) + (noc,1,),dtype=dtype)
x = tf.linalg.matmul(B,w,adjoint_a = True)
y1 = AInv.qinv(x)
y2 = fctr.QInv_auto(AInv.dhmul,rho)(x)

model1 = ppg.Model_record_grad(w,y2)
model1.compile(loss = tf.keras.losses.MSE,run_eagerly=True,optimizer=tf.keras.optimizers.SGD())
model1.fit(x=wt,y=yt,batch_size=10,epochs=8,shuffle=False,validation_data = (wv,yv))
model2 = ppg.Model_passenger(model1.gradient_record,w,y1)
model2.compile(loss = tf.keras.losses.MSE,run_eagerly=True,optimizer=tf.keras.optimizers.SGD())
model2.fit(x=wt,y=yt,batch_size=10,epochs=8,shuffle=False,validation_data = (wv,yv))
maxError = 0.
maxGrad = 0.
for grad1,grad2 in zip(model1.gradient_record,model2.gradient_record):
    for gradval1,gradval2 in zip(grad1,grad2):
        maxError = max(maxError,tf.math.reduce_max(tf.math.abs((gradval1 - gradval2)*tf.math.conj(gradval1 - gradval2))))
        maxGrad = max(maxGrad,tf.math.reduce_max(tf.math.abs(gradval1*tf.math.conj(gradval1))),tf.math.reduce_max(tf.math.abs(gradval2*tf.math.conj(gradval2))))

print(maxGrad)
print(maxError)

Epoch 1/8
10/10 [==============================] - 0s 13ms/step - loss: 5.2275 - val_loss: 5.7447
Epoch 2/8
10/10 [==============================] - 0s 12ms/step - loss: 5.2273 - val_loss: 5.7446
Epoch 3/8
10/10 [==============================] - 0s 12ms/step - loss: 5.2272 - val_loss: 5.7444
Epoch 4/8
10/10 [==============================] - 0s 12ms/step - loss: 5.2270 - val_loss: 5.7442
Epoch 5/8
10/10 [==============================] - 0s 14ms/step - loss: 5.2268 - val_loss: 5.7441
Epoch 6/8
10/10 [==============================] - 0s 13ms/step - loss: 5.2267 - val_loss: 5.7439
Epoch 7/8
10/10 [==============================] - 0s 13ms/step - loss: 5.2265 - val_loss: 5.7438
Epoch 8/8
10/10 [==============================] - 0s 13ms/step - loss: 5.2263 - val_loss: 5.7436
Epoch 1/8
10/10 [==============================] - 0s 18ms/step - loss: 5.2276 - val_loss: 5.7449
Epoch 2/8
10/10 [==============================] - 0s 15ms/step - loss: 5.2276 - val_loss: 5.7449
Epoch 3/8
10/10 [===